# Imports

In [ ]:
import sys
import os
import io, contextlib

In [ ]:
# install in your environment; do only once
!{sys.executable} -m pip install langchain openai
!{sys.executable} -m pip install langchain_community openai
!{sys.executable} -m pip install langchain_experimental
!{sys.executable} -m pip install --upgrade langchain langchain_community openai
!{sys.executable} -m pip install --upgrade langchain langchain-openai openai
!{sys.executable} -m pip install -U langchain-ollama
!{sys.executable} -m pip install tabulate

In [ ]:
from langchain_community.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [ ]:
from dsi.dsi import DSI

## AI Helper Functions

In [ ]:
# a function to ask chatgpt for queries
def ask_chatgpt_for_sql(store, user_request, open_ai_key, model_name="gpt-5"):
    # Get schema
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        if store.backend_name == 'sqlite':
            store.query("SELECT name, type, sql FROM sqlite_master WHERE sql NOT NULL ORDER BY type, name")
        else:
            store.query("SELECT table_name AS name, table_type AS type FROM information_schema.tables WHERE table_schema = 'main' ORDER BY table_type, table_name")
    query_output_str = buf.getvalue()
    schema = query_output_str.rsplit("|", 1)[-1].strip()
    
    # Set key
    os.environ["OPENAI_API_KEY"] = open_ai_key
    
    # Use LLM to extract queries
    prompt_template = (
        "You are an expert in SQL assistant.\n"
        "Here is the schema:\n{schema}\n\n"
        "Generate a SQL query for the following request:\n{user_request}\n\n"
        "Only output the SQL query on one line please."
    )
    
    prompt = PromptTemplate(
        input_variables=["schema", "user_request"],
        template=prompt_template
    )
    
    # Initialize the LLM
    llm = ChatOpenAI(model_name=model_name, temperature=0)
    
    # Create the chain
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({
        "schema": schema, 
        "user_request": user_request
    })

    print(response)
    return response.strip()

In [ ]:
def ask_ollama_for_sql(store, user_request, model_name="mistral:latest"):
    # Get schema
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        if store.backend_name == 'sqlite':
            store.query("SELECT name, type, sql FROM sqlite_master WHERE sql NOT NULL ORDER BY type, name")
        else:
            store.query("SELECT table_name AS name, table_type AS type FROM information_schema.tables WHERE table_schema = 'main' ORDER BY table_type, table_name")
    query_output_str = buf.getvalue()
    schema = query_output_str.rsplit("|", 1)[-1].strip()

    # Use LLM to extract queries
    prompt_template = (
        "You are an expert SQL assistant.\n"
        "Here is the schema:\n{schema}\n\n"
        "Generate a SQL query for the following request:\n{user_request}\n\n"
        "Only output the SQL query on one line please."
    )

    prompt = PromptTemplate(
        input_variables=["schema", "user_request"],
        template=prompt_template
    )

    # Initialize the local chat model served by Ollama
    llm = ChatOllama(model=model_name, temperature=0)

    # LCEL chain
    chain = prompt | llm | StrOutputParser()

    # NOTE: your original snippet had {user_query}; use user_request instead.
    response = chain.invoke({
        "schema": schema,
        "user_request": user_request
    })

    print(response)
    return response.strip()

# Load the data

In [ ]:
store = DSI(backend_name = "Sqlite")

In [ ]:
store.read("wildfire/wildfiredata.csv", 'CSV', table_name='wildfiredata')
store.summary()

# Use AI to extract data

## Use AI for SQL Queries

In [ ]:
user_question = "Find all entries where wind speed is above 10"
sql_query_1 = ask_chatgpt_for_sql(store, user_question, os.getenv("OPENAI_API_KEY"),  model_name="gpt-5")

In [ ]:
store.query(sql_query_1)

In [ ]:
user_question = "Find all entries where wind speed is above 10 and show it in ascending order of burned area size, and show only the top 5"
sql_query_2 = ask_ollama_for_sql(store, user_question)

In [ ]:
store.query(sql_query_2)

## Use Generic AI

In [ ]:
df = store.get_table("wildfiredata", collection = True)

In [ ]:
agent = create_pandas_dataframe_agent(
    OpenAI(temperature=0), 
    df, 
    verbose=False,
    allow_dangerous_code=True) # the real data is in the database anyway

In [ ]:
agent.invoke("how many wind speeds are above 10?")

In [ ]:
%matplotlib inline
agent.invoke("plot a scatter graph or wind_speed vs safe_unsafe_fire_behavior")